In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from tqdm import tqdm
import time
import math

In [2]:
service = Service(executable_path="./geckodriver")
driver = webdriver.Firefox(service=service)
driver.get("https://animepahe.ru")

In [3]:
# input = driver.find_element(By.CLASS_NAME, "input-search")
# input.send_keys("One piece")

In [4]:
# Get search results
# search_result_wrap = driver.find_element(By.CLASS_NAME, "search-results-wrap")

# while True:
#     try:
#         ul =  search_result_wrap.find_element(By.CLASS_NAME, "search-results")
#         li = ul.find_elements(By.TAG_NAME, "li")
#         for anime in li:
#             a = anime.find_element(By.TAG_NAME, "a")
#             result_title = a.find_element(By.CLASS_NAME, "result-title")
#             print (result_title.text.strip())
#         break
#     except Exception as e:
#         print (f"Couldn't find the results. Retrying in 5s: {e}")
#         time.sleep(5)

In [5]:
import requests

response = requests.get("https://animepahe.ru/api?m=search&q=One piece")
jsonres = None

if response.status_code == 200:
    # Print the response content (usually in JSON format for APIs)
    jsonres = response.json()
    print(jsonres)
else:
    # Print an error message if the request was not successful
    print(f"Error: {response.status_code} - {response.text}")

{'total': 61, 'per_page': 8, 'current_page': 1, 'last_page': 8, 'from': 1, 'to': 8, 'data': [{'id': 4, 'title': 'One Piece', 'type': 'TV', 'episodes': 0, 'status': 'Currently Airing', 'season': 'Fall', 'year': 1999, 'score': 8.71, 'poster': 'https://i.animepahe.ru/posters/355e6e3127aa31f0d806114169b52c4fb6da4b87df7f9c1809b9e3de97b8aac5.jpg', 'session': '33f21cc7-9761-8d35-078f-cb8a0c2e2dfa'}, {'id': 3778, 'title': 'One Piece Movie 01', 'type': 'Movie', 'episodes': 1, 'status': 'Finished Airing', 'season': 'Spring', 'year': 2000, 'score': 7.09, 'poster': 'https://i.animepahe.ru/posters/239bba86be7f81f2f7bf011f997136f67cecfa0fcaaac4b0747cdadefb45e845.jpg', 'session': 'dff46028-d546-13e3-ea0c-073d849636f1'}, {'id': 3838, 'title': 'One Piece Film: Gold - Cine Mike Popcorn Kokuchi', 'type': '', 'episodes': 1, 'status': 'Finished Airing', 'season': 'Spring', 'year': 2016, 'score': 6.36, 'poster': 'https://i.animepahe.ru/posters/7eaa1bebf5c244a8a34e95c0d3eb45d3cc0801f55c8f0e194e8b9934e8e228a0

In [19]:
def get_episode_count(session):
    driver.get(f"https://animepahe.ru/anime/{session}")
    time.sleep(1)
    episode_text = driver.find_element(By.CLASS_NAME, "episode-count").text.strip()
    return int(episode_text.split(" ")[1][1:-1])

get_episode_count(jsonres['data'][3]['session'])

1

In [22]:
# getting the episodes
def get_episode_link(episode, total):
    for i in range(0, math.ceil(total/30)):
        beginning = total - (i * 30)
        end = total - ((i * 30) + 29)
        if end <= episode and episode <= beginning:
            driver.get(f"https://animepahe.ru/anime/{jsonres['data'][0]['session']}?page={i+1}")
            time.sleep(1)
            a = driver.find_elements(By.CSS_SELECTOR, ".episode-list-wrapper .play")[beginning - episode]
            return a.get_attribute("href")
        
link = get_episode_link(500, 1088)
link

'https://animepahe.ru/play/33f21cc7-9761-8d35-078f-cb8a0c2e2dfa/6d289398f46ba5fcf2f60aa6c8bb6014b6297b07a5d7197c8403e9fb49e69d55'

In [24]:
driver.get(link)

In [25]:
# Get quality lists
qualities = []

for a in driver.find_elements(By.CSS_SELECTOR, "#pickDownload .dropdown-item"):
    quality = {
        "name": a.get_attribute("innerHTML").split(" ")[2],
        "link": a.get_attribute("href")
    }
    qualities.append(quality)

qualities

[{'name': '720p', 'link': 'https://pahe.win/FiTdn'},
 {'name': '1080p', 'link': 'https://pahe.win/KiMsh'}]

In [26]:
qualities[0]['link']

'https://pahe.win/FiTdn'

In [27]:
for quality in qualities:
    try:
        driver.get(qualities[0]['link'])
        break
    except:
        print(quality['name'], "failed trying another")

In [28]:
# Get the continue button
link_to_download = ""

while True:
    try:
        a = driver.find_element(By.XPATH, "//a[text()='Continue']")
        link_to_download = a.get_attribute("href")
        break
    except:
        print("Couldn't find button, waiting 7s")
        time.sleep(7)
 
link_to_download

'https://kwik.cx/f/dmX63ZGIh59I'

In [29]:
driver.get(link_to_download)

In [30]:
# Find button and click it
link = driver.find_element(By.CSS_SELECTOR, ".main .download form").get_attribute("action")
token = driver.find_element(By.CSS_SELECTOR, ".main .download form input").get_attribute("value")
print("Link:", link)
print("Token:", token)


Link: https://kwik.cx/d/dmX63ZGIh59I
Token: fA7hBUlb3t4deY4LcTIP7BKkcole4i8qmxHZH2LJ


In [32]:
# cookies = {}
# for cookie in driver.get_cookies():
#     cookies[cookie['name']] = cookie['value']
# cookies

cookies = '; '.join([
    f"{cookie['name']}={cookie['value']}"
    for cookie in driver.get_cookies()
])
cookies

'kwik_session=eyJpdiI6Ikk3amNRam1VZ3A2MTNxbnRTdVdJaXc9PSIsInZhbHVlIjoiWWRHSmFHQUFvZE9mL09VZ2Z0TlBxUFoxNCtWYnRVOE50dUJMTEtTZVVsb2NpN2hVaGZ1TkJLZ3YrcHk1aW5GMHlwd1ZmaE1VVXNhekc1c0RsdWJrNnlLWDlDK0lYUUl6TU1GTTJjU05jM2lvd1BSU3Q5SGlWTEovNXdSZ3ExKzMiLCJtYWMiOiJiNzZmNzU2MmUyYjkxZjE2ZTk4NzlmNWJiZWI2OWQ3OWVjOGE3ODk1NzZhNmM3MWM3MGEzOTFmZTQ2MmM2ODk0IiwidGFnIjoiIn0%3D; srv=s0; ppu_show_on_4e5e04716f26fd21bf611637f4fb8a46=1; sb_page_e1010ee4b61613b1b253d71d1c531c2e=1; sb_main_e1010ee4b61613b1b253d71d1c531c2e=1; sb_count_e1010ee4b61613b1b253d71d1c531c2e=1; sb_onpage_e1010ee4b61613b1b253d71d1c531c2e=1; dom3ic8zudi28v8lr6fgphwffqoz0j6c=7e97d99d-2670-46a2-81a3-81597cf42f49%3A1%3A1; ppu_main_4e5e04716f26fd21bf611637f4fb8a46=1; ppu_exp_4e5e04716f26fd21bf611637f4fb8a46=1704568478913; pbpr0tpuw4isk85t8yg3jb2lj5vqf=bakeryflowerbed.com'

In [33]:
data = {
    "_token": token
}
headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0',
  'Referer': 'https://kwik.cx/f/r6gCLSqIe9Nh',
  'Origin': 'https://kwik.cx',
  'Host': 'kwik.cx',
  'Cookie': cookies
}

In [34]:

data = {
    "_token": token
}
response = requests.post(link, data=data, headers=headers, stream=True)
file_name = response.headers.get('Content-Disposition').split('=')[1]
print(file_name)

response.raise_for_status()

total_size = int(response.headers.get('content-length', 0))
block_size = 1024  # 1 Kibibyte

progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

with open(file_name, 'wb') as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)

progress_bar.close()

AnimePahe_One_Piece_-_500_720p_HorribleSubs.mp4


 23%|████████▋                            | 21.6M/92.0M [00:07<00:30, 2.31MiB/s]

KeyboardInterrupt: 

 24%|████████▊                            | 22.0M/92.0M [00:20<00:30, 2.31MiB/s]